In [1]:
from neqsim import jNeqSim
import pandas as pd
import warnings
import numpy as np
# Suppress runtime warnings
warnings.filterwarnings("ignore")
from scipy.optimize import bisect
jNeqSim.util.database.NeqSimDataBase.replaceTable('COMP', "/workspaces/SolubilityCCS/Database/COMP.csv") ##

from neqsim_functions import get_water_fugacity_coefficient, get_acid_fugacity_coeff
from sulfuric_acid_activity import calc_activity_water_h2so4

In [2]:
class Phase:
    def __init__(self):
        self.components = []
        self.pressure = np.nan
        self.temperature = np.nan
        self.database = np.nan
        self.reading_properties = np.nan
        self.flow_rate = 1e-10
        self.fractions = []
        self.fraction = np.nan
        self.name = "None"

    def phase_to_fluid(self):

        fluid = Fluid()
        for component, fraction in zip(self.components, self.fractions):
            fluid.add_component(component, fraction)
        fluid.set_temperature(self.temperature)
        fluid.set_pressure(self.pressure)
        fluid.set_flow_rate(self.get_flow_rate("kg/hr"), "kg/hr")

        return fluid

    def set_phase(self, components, fractions, fraction, name):
        self.components = components
        self.fractions = fractions
        self.fraction = fraction
        self.name = name

    def set_database(self, database):
        self.database = database

    def set_properties(self, reading_properties):
        self.reading_properties = reading_properties

    def set_pressure(self, pressure):
        self.pressure = pressure

    def set_temperature(self, temperature):
        self.temperature = temperature

    def get_phase_fraction(self):
        return self.fraction

    def get_fractions(self):
        return self.fractions

    def get_component_fractions(self):
        return dict(zip(self.components, self.fractions))

    def set_phase_flow_rate(self, total_flow_rate):
        self.flow_rate = total_flow_rate * self.fraction

    def get_molar_mass(self):
        self.MW = 0
        for i, component in enumerate(self.components):
            self.MW += self.reading_properties["M"][i]*self.fractions[i]/1000
        return self.MW

    def get_fraction_component(self, component):
        for i, componenti in enumerate(self.components):

            if component == componenti:
                return self.fractions[i]
        return 0

    def get_flow_rate(self, unit):
        if unit == "mole/hr":
            return self.flow_rate
        elif unit == "kg/hr":
            return self.flow_rate*self.get_molar_mass()
        else:
            raise ValueError("No UNIT FOUND for Flow Rate")

    def get_component_flow_rate(self, component, unit):
        index = self.components.index(component)
        if unit == "mole/hr":
            return self.flow_rate*self.fractions[index]
        elif unit == "kg/hr":
            return self.get_component_flow_rate(component, "mole/hr")*self.reading_properties["M"][index]/1000
        else:
            raise ValueError("No UNIT FOUND for Flow Rate")  
            
    def get_phase_flow_rate(self, unit):
        flow_rate = 0
        for component in self.components:
            flow_rate = flow_rate + self.get_component_flow_rate(component, "kg/hr")
        return flow_rate
        
    def get_acid_wt_prc(self, name):
        acid = self.get_component_flow_rate(name, "kg/hr")
        phase_rate = self.get_phase_flow_rate("kg/hr")
        return 100*acid/phase_rate
        
    def normalize(self):
        faktor = 1 / sum(self.fractions)
        for i in range(len(self.fractions)):
            self.fractions[i] = self.fractions[i]*faktor

class Fluid:

    def __init__(self):
        self.phases = []
        self.components = []
        self.fractions = []
        self.molecular_weight = []
        self.critical_temperature = []
        self.critical_pressure = []
        self.accentric_factor = []
        self.volume_correction = []
        self.reduced_temperature = []
        self.reduced_pressure = []
        self.K_values = []
        self.m = []
        self.flow_rate = 1
        self.use_volume_correction = False
        self.AntoineParameterA = []
        self.AntoineParameterB = []
        self.AntoineParameterC = []
        self.AntoineParameterUnit = []
        self.ActivityK1 = []
        self.ActivityK2 = []
        self.ActivityK3 = []
    
        self.tol = 1e-3

        self.betta = np.nan
        self.m = []
        self.alpha = []
        self.a = []
        self.b = []
        self.A = []
        self.B = []

        self.gas_phase = Phase()
        self.liquid_phase = Phase()

        self.phases.append(self.gas_phase)
        self.phases.append(self.liquid_phase)

        self.reading_properties = {
            "M": self.molecular_weight,
            "Tc": self.critical_temperature,
            "Pc": self.critical_pressure,
            "w": self.accentric_factor,
            "s": self.volume_correction,
            "A": self.AntoineParameterA, 
            "B": self.AntoineParameterB,
            "C": self.AntoineParameterC,
            "UnitAnt": self.AntoineParameterUnit,
            "ActivityK1": self.ActivityK1,
            "ActivityK2": self.ActivityK2,
            "ActivityK3": self.ActivityK3 
        }
        for i in range(len(self.phases)):
            self.get_phase(i).set_properties(self.reading_properties)

        self.temperature = 273.15
        self.pressure = 1.01325


        self.properties = pd.read_csv("/workspaces/SolubilityCCS/Database/Properties.csv", sep=";", index_col="Component")


    def set_temperature(self, temperature):
        self.temperature = temperature
        for i in range(len(self.phases)):
            self.get_phase(i).set_temperature(temperature)

    def set_pressure(self, pressure):
        self.pressure = pressure
        for i in range(len(self.phases)):
            self.get_phase(i).set_pressure(pressure)

    def get_molar_mass(self):
        self.MW = 0
        for i, component in enumerate(self.components):
            self.MW += self.reading_properties["M"][i]*self.fractions[i]/1000
        return self.MW

    def set_flow_rate(self, flow_rate, unit):
        self.normalize()
        if unit == "mole/hr":
            self.flow_rate = flow_rate
        elif unit == "kg/hr":
            self.flow_rate = flow_rate/self.get_molar_mass()
        else:
            self.flow_rate = np.nan
            raise ValueError("No UNIT FOUND for Flow Rate")


    def get_flow_rate(self, unit):
        if unit == "mole/hr":
            return self.flow_rate
        elif unit == "kg/hr":
            return self.flow_rate*self.get_molar_mass()
        else:
            raise ValueError("No UNIT FOUND for Flow Rate")

    def read_property(self, component):
        for column_name, property in self.reading_properties.items():
            if component in self.properties.index:
                try:
                    property.append(float(self.properties.loc[component, column_name].replace(',', '.')))
                except:
                    property.append(self.properties.loc[component, column_name])
            else:
                raise ValueError(f"Properties for component {component} not found in the database.")

    def add_component(self, component, fraction):
        self.components.append(component)
        self.fractions.append(fraction)
        self.read_property(component)

    def calc_Rachford_Rice(self, betta):
        f = 0

        for i, component in enumerate(self.components):
            f += self.fractions[i]*(self.K_values[i]-1)/(1-betta+betta*self.K_values[i])
        return f

    def solve_Rachford_Rice(self):
        val_0 = self.calc_Rachford_Rice(0)
        val_1 = self.calc_Rachford_Rice(1)
        if val_0*val_1 > 0:
            if abs(val_0) < abs(val_1):
                self.betta = 0
            else:
                self.betta = 1
        else:
            self.betta = bisect(self.calc_Rachford_Rice, 0, 1)
            min_f = self.calc_Rachford_Rice(self.betta)
            if min_f > 1e-5:
                raise ValueError(f"Min f sol Rachford Rice min_f {min_f} ")
            if self.betta > 1:
                self.betta = 1.0
            if self.betta < 0:
                self.betta = 0.0
        return self.betta
    
    def calc_phases(self):
        yi = []
        xi = []
        for i, component in enumerate(self.components):
            yi.append(self.K_values[i]*self.fractions[i]/(1-self.betta+self.betta*self.K_values[i]))
            xi.append(self.fractions[i] / (1 - self.betta + self.betta * self.K_values[i]))
        self.get_phase(0).set_phase(self.components, yi, self.betta, "gas")
        self.get_phase(1).set_phase(self.components, xi, 1 - self.betta, "liquid")


    def normalize(self):
        faktor = 1 / sum(self.fractions)
        for i in range(len(self.fractions)):
            self.fractions[i] = self.fractions[i]*faktor

    def update_k_values_activity(self):

        for i, component in enumerate(self.components):
            faktor = self.activity[i]/self.fugacity[i]
            faktor = max(min(faktor, 1.1), 0.9)
            self.K_values[i] = self.K_values[i]*faktor
    
    def calc_vapour_pressure(self):
        self.vapour_pressure = []
        for i in range(len(self.fractions)):
            vapour_pressure = 10**(self.AntoineParameterA[i] - self.AntoineParameterB[i]/(self.AntoineParameterC[i] + self.temperature - 273.15))
            if self.AntoineParameterUnit[i] == "mmhg":
                vapour_pressure = vapour_pressure*0.00133322
            self.vapour_pressure.append(vapour_pressure)

    def calc_activity(self):
        self.activity = []
        self.activity_coefficient = []
        
        if len(self.components) == 1:
            activity = self.get_phase(1).fractions[0]*self.vapour_pressure[0]
            self.activity_coefficient.append(1)
            self.activity.append(activity) 
            return 
        
        if "H2O" not in self.components or (("HNO3" not in self.components) and ("H2SO4" not in self.components)):
            for i, component in enumerate(self.components):
                activity = self.get_phase(1).fractions[i]*self.vapour_pressure[i]
                if component == "CO2":
                   activity = 1e50
                self.activity_coefficient.append(1)
                self.activity.append(activity) 
            return 
        for i, component in enumerate(self.components):
            if component == "H2O":
                activity = 0
                if "HNO3" in self.components:
                    activity += np.exp((0.06*(self.temperature - 273.15)-13.3637)*
                (self.get_phase(1).get_fraction_component("HNO3"))**2)
                if "H2SO4" in self.components:
                    activity += (calc_activity_water_h2so4(self.temperature,
                            self.get_phase(1).get_fraction_component("H2O")))
            elif component == "HNO3":
                activity = np.exp((self.ActivityK1[i]*(self.temperature - 273.15)-self.ActivityK2[i])*
                (self.get_phase(1).get_fraction_component("H2O"))**2)
            elif component == "H2SO4":
                activity = np.exp((self.ActivityK1[i]*((self.temperature - 273.15)**2)+self.ActivityK2[i]*(self.temperature - 273.15) + self.ActivityK3[i])*
                (self.get_phase(1).get_fraction_component("H2O"))**2)   
            else:
                activity = 1e50
            self.activity.append(activity)  

        for i, component in enumerate(self.components):    
            self.activity_coefficient.append(self.activity[i])
            self.activity[i] = self.activity[i]*self.get_phase(1).fractions[i]*self.vapour_pressure[i]

    def calc_fugacicy_coefficient_neqsim_CPA(self):
        self.fug_coeff = []
        for i, component in enumerate(self.components): 
            if component == "H2O":
                fug_c = get_water_fugacity_coefficient(self.pressure, self.temperature - 273.15)[1]
            elif component == "HNO3" or component == "H2SO4":
                fug_c= get_acid_fugacity_coeff(component, self.pressure, self.temperature - 273.15)[0]
            elif component == "CO2": 
                fug_c = 1.0
            self.fug_coeff.append(fug_c)     

    def calc_fugacity_neqsim_CPA(self, fractions):
        self.fugacity = []
        for i, component in enumerate(self.components): 
            self.fugacity.append(self.fug_coeff[i]*self.pressure*fractions[i])          
                        

    def flash_activity(self):
        self.calc_vapour_pressure()
        self.normalize()
        self.K_values = [1e50, 0.005, 0.005]
        self.calc_fugacicy_coefficient_neqsim_CPA()
        iteration = 0
        while(1):
            K_old = self.K_values.copy()
            bettaOld = self.betta
            self.solve_Rachford_Rice()
            bettaNew  = self.betta
            delta_betta = abs(bettaOld - bettaNew)
            self.calc_phases()
            for i in range(len(self.phases)):
                    self.phases[i].set_phase_flow_rate(self.flow_rate)
            self.get_phase(1).fractions[0] = 1e-50
            self.get_phase(1).normalize()
            self.calc_fugacity_neqsim_CPA(self.phases[0].fractions)
            self.calc_activity()
            self.update_k_values_activity()
            K_new = self.K_values.copy()
            iteration += 1
            error = 0

            for i, component in enumerate(self.components):
                error += abs(K_new[i] - K_old[i])

            iteration += 1


            if iteration > 10000:
                break
                
            if error < self.tol:
                break

    def get_phase(self, i):
        return self.phases[i]





In [26]:
acid = "HNO3"
temperature = 40 # C
pressure = 100 # bara
fluid = Fluid()
fluid.add_component("CO2", 1.0) #mole
fluid.add_component("HNO3", 1200/1e6) #mole
fluid.add_component("H2O", 400/1e6) #mole
fluid.set_temperature(temperature+273.15)
fluid.set_pressure(pressure)
fluid.calc_vapour_pressure()
fluid.flash_activity()

print("Mole fraction of gas phase to total phase", fluid.betta)
print("wt of acid", fluid.phases[1].get_acid_wt_prc("HNO3"))
print("acid in kg", fluid.phases[1].get_flow_rate("kg/hr"))
print("water in CO2 ppm", 1e6*fluid.phases[0].fractions[2])
print("HNO3 in CO2 ppm", 1e6*fluid.phases[0].fractions[1])


Mole fraction of gas phase to total phase 1
wt of acid nan
acid in kg 0.0
water in CO2 ppm 399.36102236421726
HNO3 in CO2 ppm 1198.0830670926514


In [150]:
print(fluid.betta)

0.9987033927445736


In [122]:
print(fluid.betta)

0.9999978327014105


In [103]:
print(fluid.activity)

[99.75628027537425, 0.05335785686114149, 0.012157217008355644]


In [101]:
print(fluid.phases[0].fractions[1]*1e6)

1995.6220995179415


In [102]:
print(fluid.phases[0].fractions[2]*1e6)

441.57514673953267


In [70]:
fluid.phases[1].fractions[1]

0.48458537929171686

In [71]:
print(fluid.activity)

[72014252.12990843, 0.05356411847548529, 0.005596977797087575]


In [72]:
fluid.betta

0.5011535210996954

In [14]:
fluid.activity

[7.204090991912512e+17, 0.037526149181579024, 0.01420615082641079]

In [220]:
print(sum(fluid.phases[1].fractions))

0.27163123464599326


In [210]:
print(fluid.phases[0].fractions[2]*1e6)

442.65398180503286


In [91]:
print(fluid.phases[1].fractions)

[9.272845540236989e-07, 0.3993985052345737, 0.60060056748076]


In [54]:
print(fluid.phases[0].fractions[2]*1e6)

199.63547738340012


In [47]:
print(fluid.betta)

0.9906549898387311


In [92]:
acid = "HNO3"
acid_content = 0.4 
temperature = 50
pressure = 100
fluid = Fluid()
fluid.add_component(acid, acid_content)
fluid.add_component("H2O", 1-acid_content)
fluid.set_temperature(temperature+273.15)
fluid.set_pressure(pressure)
fluid.set_flow_rate(1, "kg/hr")
fluid.get_phase(1).components = fluid.components
fluid.get_phase(1).fractions = [acid_content, 1-acid_content]
fluid.calc_vapour_pressure()
fluid.calc_activity()
fluid.activity


[0.03268825220855635, 0.014148771242213292]

In [59]:
fluid.fugacity

[99.9438766655267, 0.02976979413502801, 0.01631164501700501]

In [ ]:
def calc_fugacity_neqsim_CPA():
    for component in fluid.components:
        if component == "H2O":
            get_water_fugacity_coefficient(pressure, temperature)

In [5]:
def get_acid_activity(acid, acid_content, pressure, temperature):
    fluid = Fluid()
    fluid.add_component(acid, acid_content)
    fluid.add_component("H2O", 1-acid_content)
    fluid.set_temperature(temperature+273.15)
    fluid.set_pressure(pressure)
    fluid.set_flow_rate(1, "kg/hr")
    fluid.get_phase(1).components = fluid.components
    fluid.get_phase(1).fractions = [acid_content, 1-acid_content]
    fluid.calc_vapour_pressure()
    fluid.calc_activity()
    activity_h2so4 = fluid.activity
    return activity_h2so4

In [84]:
acid = "HNO3"
acid_content = 0.38 #mole fraction of acid in water
pressure = 100 # bara
temperature = 50 # C

solubility_acid = get_acid_activity(acid, acid_content, pressure, temperature)[0]/(pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0])

#(pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0])
solubility_water = get_acid_activity(acid, acid_content, pressure, temperature)[1]/(pressure*get_water_fugacity_coefficient(pressure, temperature)[1])

print("Solubility of ", acid, " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_acid*1e6 , " ppm")
print("Solubility of ", "water ", " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_water*1e6 , " ppm")

Solubility of  HNO3   38.0  mole prc acid in CO2  is  1140.8339447111523  ppm
Solubility of  water    38.0  mole prc acid in CO2  is  534.467221559408  ppm


In [67]:
get_acid_activity(acid, acid_content, pressure, temperature)[0]

0.03066010618339378

In [58]:
pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0]

25.175000485067727

In [57]:
get_acid_activity(acid, acid_content, pressure, temperature)[0]

0.03268825220855635

In [11]:
acid = "H2SO4"
acid_content = 0.9 #mole fraction of acid in water
pressure = 1 # bara
temperature =  25 # C

solubility_acid = get_acid_activity(acid, acid_content, pressure, temperature)[0]/(pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0])
solubility_water = get_acid_activity(acid, acid_content, pressure, temperature)[1]/(pressure*get_water_fugacity_coefficient(pressure, temperature)[1])

print("Solubility of ", acid, " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_acid*1e6 , " ppm")
print("Solubility of ", "water ", " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_water*1e6 , " ppm")

Solubility of  H2SO4   90.0  mole prc acid in CO2  is  0.507987433737322  ppm
Solubility of  water    90.0  mole prc acid in CO2  is  5.512703020598119  ppm


In [43]:
def get_component_list(fluid):
    """
    This function takes a fluid neqsim object as input and returns a list of components in the fluid.

    Args:
    - fluid: a neqsim fluid object

    Returns:
    - components_list: a list of names of components in the fluid
    """

    number_of_components = fluid.getNumberOfComponents()
    components_list = []
    for i in range(number_of_components):
        component = fluid.getComponent(i)
        name = component.getName()
        components_list.append(name)
    return components_list

def get_gas_fug_coef(fluid1):
    fug = []
    components_list = get_component_list(fluid1)
    for component in components_list:
        fug.append(fluid1.getPhase(0).getComponent(component).getFugacityCoefficient())
    return fug

def get_fugacity(fluid1):
    fugacity = []
    components_list = get_component_list(fluid1)
    i = -1
    for component in components_list:
        i += 1
        fugacity.append(get_gas_fug_coef(fluid1)[i]*fluid1.getPressure("bara")*fluid1.getPhase(0).getComponent(component).getx())
    return fugacity

def get_acid_fugacity_coeff(acid, pressure, temperature):
    from neqsim import jNeqSim
    from neqsim.thermo import printFrame
    

    fluid1 = jNeqSim.thermo.system.SystemSrkCPAstatoil(298.15, 1.01325) ## CPA model
    fluid1.setTemperature(temperature, 'C')
    fluid1.setPressure(pressure, 'bara')
    fluid1.addComponent(acid, 1.0)
    fluid1.addComponent('water', 0.1)
    fluid1.addComponent('CO2', 1.0)
    fluid1.setMixingRule(9)
    fluid1.setMultiPhaseCheck(True)

    components_list = get_component_list(fluid1)
    test_ops = jNeqSim.thermodynamicOperations.ThermodynamicOperations(fluid1)
    test_ops.TPflash()

    if acid=="HNO3":    
        value = 0.37 #HNO3
    else:
        value = 0.08 - 0.27315*((temperature + 273.15)/273.15 - 1.0)

    (fluid1.getPhases()[0]).getMixingRule().setBinaryInteractionParameter(
                components_list.index(acid), components_list.index("CO2"), value)

    (fluid1.getPhases()[1]).getMixingRule().setBinaryInteractionParameter(
                components_list.index(acid), components_list.index("CO2"), value)

    test_ops.TPflash()

    return (get_gas_fug_coef(fluid1))

def get_water_fugacity_coefficient(pressure, temperature):

    temperature = temperature + 273.15
    fluid1 = jNeqSim.thermo.system.SystemSrkCPAstatoil(298.15, 1.01325) ## CPA model
    fluid1.setTemperature(temperature, 'K')
    fluid1.setPressure(pressure, 'bara')
    fluid1.addComponent('CO2', 110.0)
    fluid1.addComponent('water', 100.0)
    fluid1.setMixingRule(9)
    fluid1.setMultiPhaseCheck(True)

    components_list = get_component_list(fluid1)
    test_ops = jNeqSim.thermodynamicOperations.ThermodynamicOperations(fluid1)
    test_ops.TPflash()

    value = -0.28985
    valueT = -0.273

    val = value + valueT*(temperature/273.15 - 1.0)

    (fluid1.getPhases()[0]).getMixingRule().setBinaryInteractionParameter(
                components_list.index("water"), components_list.index("CO2"), val)


    test_ops.TPflash()

    return (get_gas_fug_coef(fluid1))


get_water_fugacity_coefficient(120, 50)


[0.5807104428484381, 0.20928043591876253]

In [19]:
acid = "H2SO4"
acid_content = 0.9 #mole fraction of acid in water
pressure = 100 # bara
temperature = 0 # C

solubility_acid = get_acid_activity(acid, acid_content, pressure, temperature)[0]/(pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0])

#(pressure*get_acid_fugacity_coeff(acid, pressure, temperature)[0])
solubility_water = get_acid_activity(acid, acid_content, pressure, temperature)[1]/(pressure*get_water_fugacity_coefficient(pressure, temperature)[1])

print("Solubility of ", acid, " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_acid*1e6 , " ppm")
print("Solubility of ", "water ", " ", acid_content*100, " mole prc acid in CO2", " is ", solubility_water*1e6 , " ppm")

Solubility of  H2SO4   90.0  mole prc acid in CO2  is  1.0519517719293339  ppm
Solubility of  water    90.0  mole prc acid in CO2  is  0.3485341251557645  ppm
